# Ecai experiments



In [64]:
from pathlib import Path
import json
import sys
import math
import subprocess
from collections import Counter
import matplotlib.pyplot as plt
import plotly

In [65]:
def file_pprint_json(filename):
    with open(filename, 'r') as f:
        array = json.load(f)
    print(json.dumps(array, indent=4, sort_keys=True))

In [66]:
def read_print(filename):
    q = Path(filename)
    with q.open() as f: 
        s = f.readlines()
        #print(s)

In [67]:
def json2array(filename):
    with open(filename, 'r') as f:
        array = json.load(f)
    return array

In [68]:
def pprint_json(array):
    print(json.dumps(array, indent=4, sort_keys=False))

In [69]:
def basecost(clues, constraints):
    if constraints == 1 and clues == 0:
        return 0
    elif constraints > 1 and clues == 0:
        return 20
    else:
        return 20 * clues    

In [70]:
def f(clues, assumptions, constraints):
    c_a = 5
    return basecost(clues, constraints) +  assumptions + c_a * (constraints + clues)

In [71]:
def f_cost(assumptions, clue, cost):
    n_assumptions = len(assumptions)
    n_clues = 0
    n_constraints = 0
    n_implicit = 0

    if clue in ["Combination of logigram constraints"]:
        n_clues = 0
        n_constraints = 2
    elif clue in ["Bijectivity", "Transitivity constraint"]:
        n_clues = 0
        n_constraints = 1
    else:
        n_clues = 1
    
    n_implicit = max(0, cost - n_assumptions - basecost(n_clues, n_constraints) - n_clues)

    updated_cost = f(n_clues, n_assumptions, n_implicit)

    return updated_cost

In [72]:
def update_cost_explanations(explanation_array):
    s_nested_explanation = "nested_explanations"
    s_reason_sequence = "reason_sequence"

    for expl in explanation_array:
        if "Solution!" in expl["clue"]:
            continue
        expl["cost"] = f_cost(assumptions = expl["assumptions"], clue = expl["clue"], cost = int(expl["cost"]) )

        if s_nested_explanation in expl:
            for n_expl in expl[s_nested_explanation]:
                if s_reason_sequence in n_expl:
                    for r_expl in n_expl[s_reason_sequence]:
                        r_expl["cost"] = f_cost(assumptions = r_expl["assumptions"], clue = r_expl["clue"], cost = int(r_expl["cost"]) )


    return explanation_array

In [73]:
def filter_nested_explanations(explanation_array):
    '''Filters the explanations with nested explanations and for every nested explanations
    removes last step from reason_sequence and if nested_explanation is only 1 step it is removed
    as well.
    Remove also last step from nested explanation list.
    '''
    s_nested_explanation = "nested_explanations"
    # s_reason_sequence = "reason_sequence"
    # data_dict = {}
    
    # take explanations only with nested explanations
    expl_w_nested = [expl for expl in explanation_array if s_nested_explanation in expl]
    for expl in expl_w_nested:
        expl[s_nested_explanation] = expl[s_nested_explanation][:-1]

    
    return expl_w_nested

In [61]:
def build_statistics(explanation_array):

    stat_dict = {
        'n_nested_expl' = [],
        ''
    }

    for expl in explanation_array:




    return stat_dict

In [62]:
'''
dict['filename'] = {costs, clue, assumptions, derivations}

    data = {}
    
    data['costs'] = []
    data['clue'] = []
    data['assumptions'] =  []
    data['derivations'] =  []

    stats['cost'] = {}
    
    stats['cost']['costs_counter']
    stats['cost']['total_cost']
    stats['cost']['avg_cost']
    
    stats['clues'] = {}
    stats['clues']['clues_counter']
    stats['clues']['clues_successive_count']
'''
results = "./output"
p = Path(results)

ignored_files = ['p12.output.json', 'p13.output.json']
files = [x for x in p.iterdir() if x.is_file() and x.name not in ignored_files]

json_file = list(filter(lambda x: 'output.json' in x.name , files) )
#problems = list(filter(lambda x: 'nielspasta' in x.name or 'p5' in x.name , json_file))
file_paths = list(map(lambda f: f.absolute().as_posix(), json_file))
#print(file_paths)

#statistics = {}
nested_data = {}
statistics = {}


for i, fp in enumerate(file_paths):
    problem_name = json_file[i].name

    json_array = json2array(fp)
    
    explanation_array = update_cost_explanations(json_array)
    nested_data[problem_name] =  filter_nested_explanations(json_array)
    statistics[problem_name] = build_statistics(nested_data[problem_name])
    #build_statistics(data[problem_name])
    # print(problem_name)
    # pprint_json(data[problem_name])
#    statistics[problem_name] = build_statistics(data[problem_name])


In [ ]:
[]